Import all required libraries and modules

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import os

In [2]:
def conditions_data(df):
    conditions = (
        df.groupby(['SURFACE_COND_DESC', 'ATMOSPH_COND_DESC', 'ROAD_GEOMETRY_DESC', 'AGE_GROUP_MODE','LIGHT_CONDITION'])
        .size()
        .reset_index(name='condition')
    )
    return conditions

In [10]:
def clustering():
    df = pd.read_csv('datasets/accident_clean.csv')
    
    crash_counts = conditions_data(df)
    
    grouped = df.groupby([
        'SURFACE_COND_DESC',
        'ATMOSPH_COND_DESC',
        'ROAD_GEOMETRY_DESC',
        'AGE_GROUP_MODE',
        'LIGHT_CONDITION'
    ])
    
    aggregated = grouped.agg({
        'VEHICLE_AGE': 'mean',
        'TOTAL_OCCUPANTS_NORM': 'mean',
        'ACCIDENT_NO': 'count',
        'ROAD_RISK': 'mean',
        'ATM_RISK': 'mean',
        'WEATHER_RISK': 'mean'       
    }).reset_index().rename(columns={'ACCIDENT_NO': 'CRASH_COUNT'})

    features = ['VEHICLE_AGE', 'TOTAL_OCCUPANTS_NORM', 'CRASH_COUNT', 'ROAD_RISK', 'ATM_RISK', 'WEATHER_RISK']

    scaler = StandardScaler()
    normalised_features = scaler.fit_transform(features)
    
    kmeans = KMeans(n_clusters=7, random_state=42)
    aggregated['CLUSTER'] = kmeans.fit_predict(normalised_features)

    for cluster_id in range(7):
        cluster_data = aggregated[aggregated['CLUSTER'] == cluster_id]
        top10 = cluster_data.sort_values(by='CLUSTER', ascending=False).head(10)
        top10.to_csv(f'cluster_{cluster_id}.csv', index=False)
        
    return()

clustering()

ValueError: could not convert string to float: 'VEHICLE_AGE'

Now we need to normalise the data!

Cluster with KMeans, using optimal k which we found to be 7 from the elbow method. We will return top 10 rows for each cluster, setting range to 7 as k = 7.